In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [3]:
movies = pd.read_csv("archive (2)/KuaiRand-Pure/KuaiRand-Pure/data/video_features_basic_pure.csv")
movies.head()


,video_id,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,server_width,server_height,music_id,music_type,tag
0,0,7349781,NORMAL,2022-04-10,LongImport,0.0,87433.0,720.0,1280.0,9155697141,9.0,39
1,1,2103883,NORMAL,2022-04-10,Kmovie,0.0,218066.0,720.0,1280.0,6355810746,9.0,2
2,2,5067285,NORMAL,2022-04-09,ShortImport,0.0,9233.0,720.0,1280.0,6618412736,4.0,1
3,3,7048760,NORMAL,2022-04-11,Web,0.0,16433.0,720.0,1280.0,9161677205,9.0,7
4,4,8635271,NORMAL,2022-04-09,Web,0.0,38766.0,720.0,1280.0,9141092381,9.0,9


In [ ]:
# Get info about the dataset
movies.info()
# Get descriptive statistics
movies.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7583 entries, 0 to 7582
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        7583 non-null   int64  
 1   author_id       7583 non-null   int64  
 2   video_type      7583 non-null   object 
 3   upload_dt       7583 non-null   object 
 4   upload_type     7583 non-null   object 
 5   visible_status  7583 non-null   float64
 6   video_duration  7344 non-null   float64
 7   server_width    7583 non-null   float64
 8   server_height   7583 non-null   float64
 9   music_id        7583 non-null   int64  
 10  music_type      7380 non-null   float64
 11  tag             7487 non-null   object 
dtypes: float64(5), int64(3), object(4)
memory usage: 711.0+ KB


,video_id,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,server_width,server_height,music_id,music_type,tag
count,7583.000000,7.583000e+03,7583,7583,7583,7583.0,7.344000e+03,7583.000000,7583.000000,7.583000e+03,7380.000000,7487
unique,NaN,NaN,3,3,14,NaN,NaN,NaN,NaN,NaN,NaN,110
top,NaN,NaN,NORMAL,2022-04-10,LongImport,NaN,NaN,NaN,NaN,NaN,NaN,39
freq,NaN,NaN,7506,3424,2925,NaN,NaN,NaN,NaN,NaN,NaN,815
mean,3791.000000,5.137301e+06,NaN,NaN,NaN,0.0,1.086159e+05,850.029804,1140.159304,8.510535e+09,8.546341,NaN
std,2189.167878,2.715180e+06,NaN,NaN,NaN,0.0,1.054980e+05,232.489241,250.230933,1.923075e+09,1.426481,NaN
min,0.000000,0.000000e+00,NaN,NaN,NaN,0.0,5.000000e+03,270.000000,448.000000,0.000000e+00,4.000000,NaN
25%,1895.500000,2.864076e+06,NaN,NaN,NaN,0.0,3.194950e+04,720.000000,960.000000,9.141054e+09,9.000000,NaN
50%,3791.000000,5.922068e+06,NaN,NaN,NaN,0.0,8.117050e+04,720.000000,1280.000000,9.148886e+09,9.000000,NaN
75%,5686.500000,7.478544e+06,NaN,NaN,NaN,0.0,1.482285e+05,960.000000,1280.000000,9.156605e+09,9.000000,NaN


In [ ]:
# Check for missing values
movies.isnull().sum()


video_id            0
author_id           0
video_type          0
upload_dt           0
upload_type         0
visible_status      0
video_duration    239
server_width        0
server_height       0
music_id            0
music_type        203
tag                96
dtype: int64

In [ ]:
# Check columns
movies.columns


Index(['video_id', 'author_id', 'video_type', 'upload_dt', 'upload_type',
       'visible_status', 'video_duration', 'server_width', 'server_height',
       'music_id', 'music_type', 'tag'],
      dtype='object')

In [ ]:
# Fill only object (string) columns with ""
for col in ["video_type", "music_type", "tag"]:
    if new_data[col].dtype == "float64":
        new_data[col] = new_data[col].fillna("").astype(str)
    else:
        new_data[col] = new_data[col].fillna("").astype(str)

# Combine selected features into a single text field
new_data["tags"] = (
    new_data["video_type"] + " " +
    new_data["music_type"] + " " +
    new_data["tag"]
)


In [ ]:
cv = CountVectorizer(max_features=10000, stop_words='english')
vector = cv.fit_transform(new_data["tags"].values.astype("U")).toarray()
vector.shape


(7583, 40)

In [ ]:
similarity = cosine_similarity(vector)


In [ ]:
def recommend(video_id):
    index = new_data[new_data["video_id"] == video_id].index[0]
    distances = list(enumerate(similarity[index]))
    distances = sorted(distances, key=lambda x: x[1], reverse=True)
    recommended = []

    for i in distances[1:6]:  # Top 5 recommendations
        recommended.append(new_data.iloc[i[0]]["video_id"])
    
    return recommended


In [ ]:
recommend(1)  # Replace with an actual video_id from your data


[np.int64(2), np.int64(3), np.int64(4), np.int64(6), np.int64(10)]

In [ ]:
pickle.dump(new_data, open("movies_list.pkl", "wb"))
pickle.dump(similarity, open("similarity.pkl", "wb"))


In [ ]:
loaded_movies = pickle.load(open("movies_list.pkl", "rb"))
loaded_similarity = pickle.load(open("similarity.pkl", "rb"))
